In [1]:
import cv2
import random
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import tensorflow as tf

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [23]:
from sklearn import svm, metrics

In [3]:
ds_path='dataset\CFD 2.0.3 Images'
# dataset = pd.read_excel("dataset.xlsx")

In [5]:
def class_val(i):
    
    if(i == 'A' or i =='F'):
        p = 0
    elif(i == 'B' or i =='M'):
        p = 1
    elif(i == 'L'):
        p = 2
    else:
        p = 3
    return p
    

In [9]:
ds = os.listdir(ds_path)
facedata = "haarcascade_frontalface_default.xml"
cascade = cv2.CascadeClassifier(facedata)
X = []
Y = []
Gender = []
for file in ds:
    
    img_path = os.path.join(ds_path,file) 
    img_files = os.listdir(img_path)
    for imfile in img_files:
        if imfile.endswith(".jpg"):
            imfile1 = os.path.join(img_path,imfile)
            main_img = cv2.imread(imfile1)
            faces = cascade.detectMultiScale(main_img, minNeighbors=12)
            for f in faces:
                x, y, w, h = [ v for v in f ]
                cv2.rectangle(main_img, (x,y), (x+w,y+h), (255,255,255))
                sub_face = main_img[y:y+h, x:x+w]
                face_file_name = "dataset\\"+ file +".jpg"
                
                sub_face = cv2.resize(sub_face, (128,128),interpolation=cv2.INTER_AREA)
                img = cv2.cvtColor(sub_face,cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                
                vertical_img = cv2.flip(gray, 1)
                
                X.append(gray)
                X.append(vertical_img)
                Y.append(class_val(file[0]))
                Y.append(class_val(file[0]))
#                 class_cnn(file[0])
#                 Gender.append(class_val(file[1]))
#                 Gender.append(class_val(file[1]))
#                 face_flip = "dataset\\"+file+"1.jpg"
#                 X.append()
#                 cv2.imwrite(face_file_name, gray)
#                 cv2.imwrite(face_flip,vertical_img)

#             
#             vertical_img = cv2.flip(main_img, 1 )


In [11]:
keep_probability = 0.7
epochs = 10
batch_size = 50
num_classes = 4
learning_rate = 0.00125

In [12]:
data_cnn = X
class_cnn = Y
# bb = Gender
data_cnn1 = np.zeros((len(data_cnn),64,64))
for i in range(len(data_cnn)):
    data_cnn1[i] = cv2.resize(data_cnn[i], (64,64),interpolation=cv2.INTER_AREA)
data_cnn1.shape
# from sklearn.utils import shuffle
# data_cnn, class_cnn = shuffle(data_cnn,class_cnn, random_state=123)
from sklearn.model_selection import train_test_split
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(data_cnn1, class_cnn, test_size=0.2, random_state=123)
X_train_cnn = X_train_cnn.reshape(1976,64,64,1)
X_test_cnn = X_test_cnn.reshape(494,64,64,1)
# len(X_test_cnn)
y_train_cnn = keras.utils.to_categorical(y_train_cnn, num_classes)
y_test_cnn = keras.utils.to_categorical(y_test_cnn, num_classes)

In [ ]:
# tf.reset_default_graph()
# x = tf.placeholder(tf.float32, shape=(None, 64,64,1), name='input_x')
# y =  tf.placeholder(tf.float32, shape=(None, 4), name='output_y')
# keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [13]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(64,64,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation(tf.nn.softmax))

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train_cnn, y_train_cnn,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_cnn, y_test_cnn))


Train on 1976 samples, validate on 494 samples
Epoch 1/10
1976/1976 [==============================] - 97s 49ms/step - loss: 9.2153 - acc: 0.4261 - val_loss: 9.3315 - val_acc: 0.4211
Epoch 2/10
1976/1976 [==============================] - 96s 49ms/step - loss: 9.1031 - acc: 0.4352 - val_loss: 9.3315 - val_acc: 0.4211
Epoch 3/10
1976/1976 [==============================] - 90s 46ms/step - loss: 9.1031 - acc: 0.4352 - val_loss: 9.3315 - val_acc: 0.4211
Epoch 4/10
1976/1976 [==============================] - 95s 48ms/step - loss: 9.1031 - acc: 0.4352 - val_loss: 9.3315 - val_acc: 0.4211
Epoch 5/10
1976/1976 [==============================] - 96s 49ms/step - loss: 9.1031 - acc: 0.4352 - val_loss: 9.3315 - val_acc: 0.4211
Epoch 6/10
1976/1976 [==============================] - 96s 49ms/step - loss: 9.1031 - acc: 0.4352 - val_loss: 9.3315 - val_acc: 0.4211
Epoch 7/10
1976/1976 [==============================] - 95s 48ms/step - loss: 9.1031 - acc: 0.4352 - val_loss: 9.3315 - val_acc: 0.4211
E

NameError: name 'x_test_cnn' is not defined

In [15]:
score = model.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 9.331529130819838
Test accuracy: 0.4210526318202617


In [16]:
Y = np.array(Y)

In [17]:
data = np.zeros((len(X),16385))
for i in range(len(X)):
    data[i,:-1] = np.reshape(X[i],16384)
    data[i, -1] = Y[i]


In [18]:
random.seed(123)
random.shuffle(data)
split = int(0.85*data.shape[0])
x_train = data[:split,:-1]
x_test = data[split:,:-1]
y_train = data[:split,-1]
y_test = data[split:,-1]


In [21]:
# SVM
classifier = svm.SVC(gamma=0.001)
classifier.fit(x_train,y_train)
y_pred_SVM = classifier.predict(x_test)

In [24]:
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(y_test, y_pred_SVM)))

Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99        43
         1.0       0.46      1.00      0.63       144
         2.0       1.00      0.16      0.28        43
         3.0       1.00      0.08      0.14       141

   micro avg       0.55      0.55      0.55       371
   macro avg       0.87      0.55      0.51       371
weighted avg       0.79      0.55      0.45       371




In [25]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred_SVM))

Confusion matrix:
[[ 42   1   0   0]
 [  0 144   0   0]
 [  0  36   7   0]
 [  0 130   0  11]]


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_SVM)

0.5498652291105122

In [28]:
class Neighbor(object):
    def __init__(self):
        pass

    def train(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        test = X_test.shape[0]
        y_pred = np.zeros(test, dtype = self.y_train.dtype)
        for i in range(test):
            distance = np.sum(np.abs(self.X_train - X_test[i,:]), axis = 1)
            ind = np.argmin(distance)
            y_pred[i] = self.y_train[ind]

        return y_pred

In [29]:
nn = Neighbor()
nn.train(x_train, y_train)
y_pred = nn.predict(x_test)
accuracy_score(y_test, y_pred)


0.7735849056603774

In [31]:
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(y_test, y_pred)))

Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
              precision    recall  f1-score   support

         0.0       0.44      1.00      0.61        43
         1.0       0.94      1.00      0.97       144
         2.0       0.60      0.63      0.61        43
         3.0       0.97      0.52      0.68       141

   micro avg       0.77      0.77      0.77       371
   macro avg       0.74      0.79      0.72       371
weighted avg       0.85      0.77      0.77       371




In [30]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Confusion matrix:
[[ 43   0   0   0]
 [  0 144   0   0]
 [  9   5  27   2]
 [ 45   5  18  73]]


In [ ]:
# def cnn(x, keep_prob):
#     conv1_filter = tf.Variable(tf.truncated_normal(shape=[3,3, 1, 64], mean=0, stddev=0.08))
#     conv2_filter = tf.Variable(tf.truncated_normal(shape=[3,3, 64, 128], mean=0, stddev=0.08))
    
#     conv1 = tf.nn.conv2d(x, conv1_filter, strides=[1,1,1,1], padding='SAME')
#     conv1 = tf.nn.relu(conv1)
#     conv1_pool = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#     conv1_bn = tf.layers.batch_normalization(conv1_pool)
    
#     conv2 = tf.nn.conv2d(conv1_bn, conv2_filter, strides=[1,1,1,1], padding='SAME')
#     conv2 = tf.nn.relu(conv2)
#     conv2_pool = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
#     conv2_bn = tf.layers.batch_normalization(conv2_pool)
    
#     flat = tf.contrib.layers.flatten(conv2_bn)  
    
#     full = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=1024, activation_fn=tf.nn.relu)
#     full = tf.nn.dropout(full, keep_prob)
#     full = tf.layers.batch_normalization(full)        
    
#     out = tf.contrib.layers.fully_connected(inputs=full, num_outputs=4, activation_fn=None)
#     return out

In [ ]:
# logits = cnn(x, keep_prob)

# # Loss and Optimizer
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# # Accuracy
# correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

# def train_neural_network(session, optimizer, keep_probability, X, y):
#     session.run(optimizer, 
#                 feed_dict={
#                     x: X,
#                     y: y,
#                     keep_prob: keep_probability
#                 })
    
# with tf.Session() as sess:
#     # Initializing the variables
#     sess.run(tf.global_variables_initializer())
    
#     # Training cycle
#     train_neural_network(sess, optimizer, keep_probability, X_train_cnn, y_train_cnn)
#     print_stats(sess, batch_features, batch_labels, cost, accuracy)
    


In [ ]:
# max_count=100
# reg_val=[]
# lr_val=[]
# test_loss=[]
# test_acc=[]

# for i in range(max_count):
#     reg=10**(np.random.uniform(-4,0) )
#     lr=10**(np.random.uniform(-3,-4) )

# #Defining the architechture

# model=models.Sequential()

# model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(60,60,3)))

# model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.Conv2D(64,(3,3),activation='relu'))

# model.add(layers.MaxPooling2D(2,2))

# model.add(layers.Conv2D(128,(3,3),activation='relu'))

# model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.Conv2D(128,(3,3),activation='relu'))

# model.add(layers.MaxPooling2D((2,2)))

# model.add(layers.Flatten())

# model.add(layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l2(reg)))

# model.add(layers.Dense(1,activation='sigmoid',kernel_regularizer=regularizers.l2(reg)))

# train_generator=train_datagen.flow_from_directory(
#                 train_dir,
#                 target_size=(60,60),
#                 batch_size=20,
#                 class_mode='binary')
# test_generator=test_datagen.flow_from_directory(
#                 test_dir,
#                 target_size=(60,60),
#                 batch_size=20,
#                 class_mode='binary'
#                 )
# history=model.fit_generator(train_generator,steps_per_epoch=100,epochs=5,validation_data=test_generator,validation_steps=50)

# reg_val.append(reg)
# lr_val.append(lr)
# test_loss.append(history.history['val_loss'])
# test_acc.append(history.history['val_acc'])

# print ("*"*30)
# print ("Finding the highest Test Accuracy and lowest Test Loss...")

# index1=0
# index2=0
# max_test_acc=max(test_acc[0])
# min_test_loss=min(test_loss[0])
# for i in range(max_count):
# temp1=max(test_acc[i])
# if(temp1>=max_test_acc):
#     max_test_acc=temp1
#     index1=i
#     temp2=min(test_loss[i])
# if(temp2<min_test_loss):
#     min_test_loss=temp2
#     index2=i 

# print ('Maximum Testing Accuracy:',max_test_acc)
# print ('Minimum Testing Loss:',min_test_loss)
# print ('Value of optimum learning rate :',lr_val[index1])
# print ('Value of optimum regularization:',reg_val[index2])

In [ ]:
# ds1 = os.listdir("dataset")
# filenames =[]
# for file in ds1:
#     if file.endswith(".jpg"):
#         filenames.append(file)
# filenames.sort()
# random.seed(123)
# random.shuffle(filenames)
# split_1 = int(0.8 * len(filenames))
# split_2 = int(0.9 * len(filenames))
# train_filenames = filenames[:split_1]
# dev_filenames = filenames[split_1:split_2]
# test_filenames = filenames[split_2:]
# #     print (len(filenames))
# y_train = []
# y_dev = []
# y_test = []
# for i in train_filenames:
#     y_train.append(class_val(i[0]))
# for j in dev_filenames:
#     y_dev.append(class_val(j[0]))
# for k in test_filenames:
#     y_test.append(class_val(k[0]))
    
# train_filenames = np.array(train_filenames)
# test_filenames =np.array(test_filenames)
# dev_filenames = np.array(dev_filenames)
# y_train = np.array(y_train)
# y_dev = np.array(y_dev)
# y_test = np.array(y_test)

# # l = ["A" = , "B", "L", "W"]



In [ ]:
# img = cv2.imread("dataset\CFD 2.0.3 Images\WM-258\CFD-WM-258-125-N.jpg")
# facedata = "haarcascade_frontalface_default.xml"
# cascade = cv2.CascadeClassifier(facedata)
# faces = cascade.detectMultiScale(img)
# for f in faces:
#     x, y, w, h = [ v for v in f ]
#     cv2.rectangle(img, (x,y), (x+w,y+h), (255,255,255))

#     sub_face = img[y:y+h, x:x+w]
#     face_file_name = "dataset\CFD 2.0.3 Images\WM-258\CFD-WM-258-125-N" + str(y) + ".jpg"
#     cv2.imwrite(face_file_name, sub_face)
# plt.imshow(img)

 
# # cv2.resize(image, (128,128),interpolation=cv2.INTER_AREA)
# # vertical_img = cv2.flip(image, 1 )
# # img = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
# # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
# # plt.imshow(vertical_img)